# Introduction

I added some comments throughout this notebook to guide you so I hope it's easily understandable. Let me know if you have any questions or comments either in person or shoot me an e-mail (jozbus@ethz.ch) or slack message (@Joost). Shout out to Quentin Ficheux for much of the framework.

Joost Bus (2021/12/01)

# Imports and directories

Before you start, make sure you merged `feature/hamfit_analysis` into your current branch (but since you have this notebook, you probably already have)

In [ ]:
import numpy as np
from pycqed.utilities.hamiltonian_fitting_analysis import HamiltonianFittingAnalysis as HF

import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = [12, 8]

# Loading relevant data

Example data taken on the ATC75_M136_S17HW02 device in November 2021

## Model data

Defining the model data. It is grouped per voltage, then transition and then that transitions frequency, see below for an example.

In [ ]:
# characterization Hamiltonian fitting

#voltages
uss_voltage = -2.316
lss_voltage = -5.651
mid_voltage = -3.9835

#ge-frequencies
uss_ge_freq = 5.887164e9
lss_ge_freq = 4.151017e9
mid_ge_freq = 5236129258

#ef-frequency
uss_ef_freq = 5.713653e9

#model values
experimental_values_model = {
    uss_voltage: {
        'ge': uss_ge_freq, # ge at lss
        'ef': uss_ef_freq, # ef at uss        
    },
    lss_voltage: {
        'ge': lss_ge_freq, # ge at lss      
    },
    mid_voltage: {
        'ge': mid_ge_freq, # ge at mid    
    },
}

Note that you are quite free to choose how to write the transitions. 

Examples:
- The ge-transition can be given as `'ge'` or `((0,0),(1,0))` or `'g,0-e,0'`
- The ef-transition can be given as `'ef'` or `((1,0),(2,0))` or `'e,0-f,0'`

## Verification data

The verification measurements are saved to a dictionary with the same format

In [ ]:
# characterization Hamiltonian fitting

# # example for format
# experimental_values_verification = {
#     0.531:{
#         ((0,0),(1, 0)):6.143162354737646e9,
#         ((1,0),(2, 0)):5.971582479578413e9,
#         ((0,0),(0, 1)):7.1543e9,
#     }
# }

experimental_values_verification = {-2.5261875000000003: {((0, 0), (1, 0)): 5876861838.594975},
 -2.734375: {((0, 0), (1, 0)): 5845556466.100952},
 -2.9425625: {((0, 0), (1, 0)): 5792983999.538286},
 -3.15075: {((0, 0), (1, 0)): 5720042348.821003},
 -3.3589375: {((0, 0), (1, 0)): 5626537070.824841},
 -3.567125: {((0, 0), (1, 0)): 5514561842.759405},
 -3.7753125: {((0, 0), (1, 0)): 5383936019.117272},
 -3.9835000000000003: {((0, 0), (1, 0)): 5236129258.242982},
 -4.1916875000000005: {((0, 0), (1, 0)): 5075064940.79929},
 -4.399875: {((0, 0), (1, 0)): 4903849522.684156},
 -4.6080625: {((0, 0), (1, 0)): 4727949872.591458},
 -4.81625: {((0, 0), (1, 0)): 4555756653.668654},
 -5.0244374999999994: {((0, 0), (1, 0)): 4398180050.318708},
 -5.2326250000000005: {((0, 0), (1, 0)): 4267995295.45334},
 -5.4408125: {((0, 0), (1, 0)): 4181664919.8280997}}

In [ ]:
experimental_values_verification

# Hamiltonian fitting

## Checking model data by plotting and printing

Quickly plotting the model data to make sure everything makes sense...

In [ ]:
HF.plot_experimental_values(experimental_values_model, print_values=True)

## Fitting

Here I define the initial guess parameters. Depending on what parameters you choose to optimize (see parameters_to_optimize variable) some are taken to be fixed. For example, `dac_sweet_spot` or `V_per_phi0` can be calculated from the sweet spot voltages instead of being fitted.

In [ ]:
# parameters to be optimized
parameters_to_optimize = ['Ej_max', 'E_c', 'asymmetry', 'coupling']

# initial parameters and fixed parameters
dac_sweet_spot = uss_voltage
V_per_phi0 = 2*np.abs(uss_voltage-lss_voltage)

p_guess ={
    'dac_sweet_spot': dac_sweet_spot,
    'V_per_phi0': V_per_phi0,
    'Ej_max': 30e9,
    'E_c': 175e6,
    'asymmetry': 0.3,
    'coupling': 0.2e9,
    'fr': 7.553e9,
}

In the next cell, the actual optimization happens

In [ ]:
f = HF.optimizer(experimental_values=experimental_values_model, 
                 parameters_to_optimize=parameters_to_optimize, 
                 parameters_guess=p_guess,
                 method='Nelder-Mead') #'Nelder-Mead'

result_dict = HF.fit_parameters_from_optimization_results(f, parameters_to_optimize, p_guess)

## Results

Showing the resulting parameters...

In [ ]:
result_dict

...and showing the model in conjunction with the experimental values...

In [ ]:
HF.plot_model_and_experimental_values(result_dict, experimental_values_model, 
                                      transitions = ['ge','ef',((0,0),(0,1))]
                                     )

## Visualizing verification measurements

Next we can plot the verification measurements, and in the cell after the model versus the verification measurements.

In [ ]:
HF.plot_experimental_values(experimental_values_verification, print_values=True)

In [ ]:
HF.plot_model_and_experimental_values(result_dict, experimental_values_verification)

## Residuals

Lastly, we show the residuals (they are returned in this function). You can also choose to plot them using the `plot_residuals=True` parameter.

In [ ]:
residuals = HF.calculate_residuals(result_dict, experimental_values_verification,
                                   plot_residuals=True)

The end